In [1]:
import numpy as np

with open('data/reviews.txt','r') as f:
    reviews = f.read()
with open('data/labels.txt','r') as f:
    labels = f.read()

In [2]:
print(reviews[:1000])
print()
print(labels[:20])

bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life  such as  teachers  . my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers  . the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the whole situation  all remind me of the schools i knew and their students . when i saw the episode in which a student repeatedly tried to burn down the school  i immediately recalled . . . . . . . . . at . . . . . . . . . . high . a classic line inspector i  m here to sack one of your teachers . student welcome to bromwell high . i expect that many adults of my age think that bromwell high is far fetched . what a pity that it isn  t   
story of a man who has unnatural feelings for a pig . starts out with a opening scene that is a terrific example of absurd comedy . a formal orchestra audience is turn

# 把所有文字都用array存

In [3]:
from string import punctuation

reviews = reviews.lower()

words = ''.join([c for c in reviews if c not in punctuation])
# words[:100]

review_split = words.split('\n')
words = ' '.join(review_split)
words = words.split()

In [4]:
words[:10]

['bromwell', 'high', 'is', 'a', 'cartoon', 'comedy', 'it', 'ran', 'at', 'the']

# 將每則評論(review)encode到數字

> padding用0 , 所以vocab to int要從1開始

In [5]:
from collections import Counter

counts = Counter(words)

# vovab_to_int
counts = sorted(counts,key=counts.get,reverse=True)
vocab_to_int = { w:i for i,w in enumerate(counts,1) }

review_to_int = []
for review in review_split:
    
    review_to_int.append([vocab_to_int[w] for w in review.split()])

In [6]:
review_to_int[:1]

[[21025,
  308,
  6,
  3,
  1050,
  207,
  8,
  2138,
  32,
  1,
  171,
  57,
  15,
  49,
  81,
  5785,
  44,
  382,
  110,
  140,
  15,
  5194,
  60,
  154,
  9,
  1,
  4975,
  5852,
  475,
  71,
  5,
  260,
  12,
  21025,
  308,
  13,
  1978,
  6,
  74,
  2395,
  5,
  613,
  73,
  6,
  5194,
  1,
  24103,
  5,
  1983,
  10166,
  1,
  5786,
  1499,
  36,
  51,
  66,
  204,
  145,
  67,
  1199,
  5194,
  19869,
  1,
  37442,
  4,
  1,
  221,
  883,
  31,
  2988,
  71,
  4,
  1,
  5787,
  10,
  686,
  2,
  67,
  1499,
  54,
  10,
  216,
  1,
  383,
  9,
  62,
  3,
  1406,
  3686,
  783,
  5,
  3483,
  180,
  1,
  382,
  10,
  1212,
  13583,
  32,
  308,
  3,
  349,
  341,
  2913,
  10,
  143,
  127,
  5,
  7690,
  30,
  4,
  129,
  5194,
  1406,
  2326,
  5,
  21025,
  308,
  10,
  528,
  12,
  109,
  1448,
  4,
  60,
  543,
  102,
  12,
  21025,
  308,
  6,
  227,
  4146,
  48,
  3,
  2211,
  12,
  8,
  215,
  23]]

# encode labels (1:positive  ,  0:negative)

In [7]:
encoded_labels = [1 if label=='positive' else 0 for label in labels.split("\n") ]

In [8]:
len(encoded_labels)

25001

# 刪一下離群值

> 刪一下長度為0的review

In [9]:

print(len(review_to_int))

none_zero_idx = [ idx for idx,review in enumerate(review_to_int) if len(review)!=0]
none_zero_review = [review_to_int[idx] for idx in none_zero_idx]

25001


In [10]:
print(len(none_zero_review))


25000


In [11]:
encoded_labels = [ v for i,v in enumerate(encoded_labels) if i in none_zero_idx]
print(len(encoded_labels))

25000


# padding seq

> 限制一個長度, 超過clip掉,太少補0在前面

In [12]:
review_to_int = np.array(none_zero_review[:])
encoded_labels = np.array(encoded_labels[:])

In [13]:
encoded_labels = encoded_labels.astype(int)

In [14]:
def padding_seq(review_int , seq_len):
    
    features = np.zeros((len(review_int),seq_len),dtype=np.int)
    for i,r in enumerate(review_int):
        features[i,-len(r):] =  np.array(r)[:seq_len]
    
    return features

In [15]:
seq_len = 200
features = padding_seq(review_to_int , seq_len)
print(features[:30,:10])

[[    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [22382    42 46418    15   706 17139  3389    47    77    35]
 [ 4505   505    15     3  3342   162  8312  1652     6  4819]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [   54    10    14   116    60   798   552    71   364     5]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    1   330   578    34     3   162   748  2731     9   325]
 [    9    11 10171  5305  1946   689   444    22   280   673]
 [    0     0     0     0     0     0     0     0     0

In [16]:
features = features.astype(int)

# 切資料集

> 80% train , 10% valid , 10% test

In [17]:
frac_ratio = 0.8
split_idx = int(len(features)*0.8)
train_x , remain_x = np.array(features[:split_idx]) , np.array(features[split_idx:])
train_y ,remain_y = np.array(encoded_labels[:split_idx]) , np.array(encoded_labels[split_idx:])

frac_ratio = len(remain_x)//2
valid_x,test_x = remain_x[:frac_ratio],remain_x[frac_ratio:]
valid_y,test_y = remain_y[:frac_ratio],remain_y[frac_ratio:]

# 建立dataloader

In [18]:
import torch
from torch.utils.data import TensorDataset , DataLoader

In [19]:
len(test_y)

2500

In [20]:
train_data = TensorDataset( torch.from_numpy(train_x), torch.from_numpy(train_y))
valid_data = TensorDataset( torch.from_numpy(valid_x), torch.from_numpy(valid_y))
test_data = TensorDataset( torch.from_numpy(test_x), torch.from_numpy(test_y))

batch_size = 50
train_loader = DataLoader(train_data ,batch_size=batch_size,shuffle=True)
valid_loader = DataLoader(valid_data ,batch_size=batch_size,shuffle=True)
test_loader = DataLoader(test_data ,batch_size=batch_size,shuffle=True)

In [21]:
sample_x,sample_y = next(iter(train_loader))
print(sample_x)
print(sample_y)

tensor([[     0,      0,      0,  ...,   6742,     45,      4],
        [    10,     84,    548,  ...,     53,      6,    894],
        [     0,      0,      0,  ...,    147,    287,    291],
        ...,
        [  1540,   1963,  25099,  ...,   1729,      3,  22398],
        [     0,      0,      0,  ...,   1918,     16,    336],
        [     0,      0,      0,  ...,      2,    440,     20]])
tensor([ 0,  1,  0,  1,  1,  1,  1,  0,  1,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  1,  1,  1,  0,  1,  0,  1,  0,  1,
         0,  0,  1,  1,  1,  0,  0,  0,  1,  1,  0,  0,  1,  1,
         0,  1,  0,  0,  1,  0,  0,  1])


# train which device

In [22]:
train_on_gpu = torch.cuda.is_available()
if train_on_gpu:
    print('train on GPU')
else:
    print('train on cpu')

train on GPU


# build RNN

In [23]:
import torch.nn as nn

In [24]:
class RNN(nn.Module):
    def __init__(self , vocab_size, output_size , embedding_dim , hidden_size , n_layer=2 , drop_prob=0.5):
        
        super(RNN,self).__init__()
        
        self.output_size = output_size
        self.embed_dim = embedding_dim
        self.n_layer = n_layer
        self.dropout = nn.Dropout(0.3)
        self.hidden_dim = hidden_size
        
        self.embedding = nn.Embedding(vocab_size,embedding_dim)
        self.lstm = nn.LSTM(embedding_dim,hidden_size,n_layer,dropout=drop_prob,batch_first = True)
        self.fc = nn.Linear(hidden_size,output_size)
        self.sig = nn.Sigmoid()
        
    def forward(self,x,hidden):
        batch_size = x.size(0)
        x = self.embedding(x)
        r_out,hidden = self.lstm(x,hidden)
        
        out = r_out.contiguous().view(-1,self.hidden_dim)
        out = self.dropout(out)
        out = self.fc(out)
        out = self.sig(out)
        
        out = out.view(batch_size,-1)
        out = out[:,-1]
        
        return out,hidden
        
        
    def init_hidden(self,batch_size):
        
        # 新增兩個tensor 給 hidden state 和 cell state
        # 兩個tensor的shape = n_layer * batch_size * hidden_dim  (順序是固定,不能變動)
        weight = next(self.parameters()).data
        
        if train_on_gpu:
            hidden = (weight.new(self.n_layer,batch_size,self.hidden_dim).zero_().cuda() , 
                     weight.new(self.n_layer,batch_size,self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layer,batch_size,self.hidden_dim).zero_(),
                     weight.new(self.n_layer,batch_size,self.hidden_dim).zero_())
        return hidden

# 初始化network

In [25]:
vocab_size = len(vocab_to_int)+1  # +1 for padding
output_size = 1   # 只輸出pos or neg
embed_dim = 400
hidden_dim = 256
n_layer = 2


net = RNN(vocab_size , output_size , embed_dim , hidden_dim , n_layer)
print(net)

RNN(
  (dropout): Dropout(p=0.3)
  (embedding): Embedding(74073, 400)
  (lstm): LSTM(400, 256, num_layers=2, batch_first=True, dropout=0.5)
  (fc): Linear(in_features=256, out_features=1, bias=True)
  (sig): Sigmoid()
)


# 訓練囉

In [26]:
lr = 0.001
epochs = 4
print_every = 100

clip = 5
counter = 0

criterion = nn.BCELoss()
opt = torch.optim.Adam(net.parameters(),lr=lr)

In [28]:
if train_on_gpu:
    net = net.cuda()
net.train()



for epoch in range(epochs):
    h = net.init_hidden(batch_size)
    
    for idx , (inputs , labels) in enumerate(train_loader):
        counter += 1
        if train_on_gpu:
            inputs = inputs.cuda()
            labels = labels.cuda()
            
            
        # 每次batch 都要create新的variables for hidden state, 不然會一直累積
        h = tuple([each.data for each in h])

        net.zero_grad()

        output , h = net(inputs , h)
        loss = criterion(output.squeeze() , labels.float())

        loss.backward()

        ## 要clip gradient ,不然會梯度爆炸
        nn.utils.clip_grad_norm_(net.parameters() , clip) 
        opt.step()

        if counter % print_every ==0:

            val_h = net.init_hidden(batch_size)
            val_losses = []
            net.eval()

            for val_inputs , val_labels in valid_loader:

                if train_on_gpu:
                    val_inputs,val_labels = val_inputs.cuda() , val_labels.cuda()
                val_h = tuple([ each.data for each in val_h ])

                val_out , val_h = net(val_inputs,val_h)
                val_loss = criterion(val_out.squeeze() , val_labels.float())
                val_losses.append(val_loss.item())


            net.train()
            print("Epoch: {}/{}".format(epoch+1,epochs),
                 "step: {}".format(counter),
                 "Loss: {:.4f}".format(loss.item()),
                 "valid_Loss {:.4f}".format(np.mean(val_losses)))
            
        
        

Epoch: 1/4 step: 500 Loss: 0.4309 valid_Loss 0.5483
Epoch: 1/4 step: 600 Loss: 0.3964 valid_Loss 0.4810
Epoch: 1/4 step: 700 Loss: 0.3857 valid_Loss 0.4888
Epoch: 1/4 step: 800 Loss: 0.2996 valid_Loss 0.4762
Epoch: 2/4 step: 900 Loss: 0.3196 valid_Loss 0.4985
Epoch: 2/4 step: 1000 Loss: 0.4524 valid_Loss 0.5489
Epoch: 2/4 step: 1100 Loss: 0.3328 valid_Loss 0.4814
Epoch: 2/4 step: 1200 Loss: 0.2761 valid_Loss 0.4586
Epoch: 3/4 step: 1300 Loss: 0.1016 valid_Loss 0.5076
Epoch: 3/4 step: 1400 Loss: 0.1097 valid_Loss 0.5038
Epoch: 3/4 step: 1500 Loss: 0.1801 valid_Loss 0.6004
Epoch: 3/4 step: 1600 Loss: 0.1585 valid_Loss 0.5138
Epoch: 4/4 step: 1700 Loss: 0.1140 valid_Loss 0.5865
Epoch: 4/4 step: 1800 Loss: 0.0709 valid_Loss 0.5724
Epoch: 4/4 step: 1900 Loss: 0.0412 valid_Loss 0.5717
Epoch: 4/4 step: 2000 Loss: 0.0966 valid_Loss 0.5821


# 跑test data囉

In [32]:

h = net.init_hidden(batch_size)
test_losses=[]
num_correct=0
net.eval()


for inputs,labels in test_loader:
    
    
    
    h = tuple([each.data for each in h])
    if train_on_gpu:
        inputs,labels = inputs.cuda(),labels.cuda()
    out , h = net(inputs,h)
    
    test_loss = criterion(out.squeeze(),labels.float())
    test_losses.append(test_loss.item())
    
    pred = torch.round(out.squeeze())
    correct_tensor = pred.eq(labels.float().view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)
    
    

print("test loss: {:.4f}".format(np.mean(test_losses)))
print("test acc: {:.4f}".format(num_correct/len(test_loader.dataset)))
    
    
    



test loss: 0.5589
test acc: 0.8032


# 來試試看打一些評論 看看model預測的結果

In [33]:
review_pos = "The movie is very good. I like this movie, it's so DOPE !!!"

In [38]:
from string import punctuation

def tokenize(review):
    review = review.lower()
    text = ''.join([c for c in review if c not in punctuation])
    
    text_arr = text.split()
    text_int = [ vocab_to_int[w] for w in text_arr]
    return [text_int]
    
    

In [39]:
test_int = tokenize(review_pos)
print(test_int)

[[1, 18, 6, 55, 50, 10, 39, 11, 18, 92, 37, 10417]]


In [41]:
seq_len = 200
pad_test = padding_seq(test_int,seq_len)
print(pad_test)

[[    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0 

In [42]:
def predict(net,test_review,seq_len=200):
    net.eval()
    test_int = tokenize(test_review)
    pad_test = padding_seq(test_int,seq_len)
    
    
    review_tensor = torch.from_numpy(pad_test)
    batch_size = review_tensor.size(0)
    
    h = net.init_hidden(batch_size)
    
    if train_on_gpu:
        review_tensor = review_tensor.cuda()
    out , h = net(review_tensor,h)
    
    pred = torch.round(out.squeeze())
    
    if pred.item()==1:
        print("review is positive !")
    else:
        print("review is negative !")
    

In [43]:
predict(net,review_pos)

review is positive !


In [44]:
review_neg = "Damn, this movie is like shit. I don't see it any more"

In [45]:
predict(net,review_neg)

review is negative !
